# AOI Embeddings Slide (Hamburg, IA) — Google Colab Notebook

This notebook builds a single slide with three panels:

- **Panel A (Global):** Global context with the AOI marked and coordinates labeled.
- **Panel B (AOI Embeddings):** Satellite Embedding RGB visualization (3 axes from the 64‑D embedding vector) clipped to the AOI.
- **Panel C (Points):** Lat/long plot styled like your figure (density shading, points colored by resilience classes), with **north arrow** and **2 km scale bar**.

**Datasets & references**:
- Satellite Embedding dataset (`GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL`), 64 bands A00..A63 (annual, 2017+). See Google Developers tutorial: 
  https://developers.google.com/earth-engine/tutorials/community/satellite-embedding-01-introduction
- Earth Engine Python API setup in Google Colab: 
  https://developers.google.com/earth-engine/guides/python_install-colab
- Hamburg, Iowa coordinates (used for labeling): 
  https://en.wikipedia.org/wiki/Hamburg,_Iowa
- Optional global true‑color backdrop (NASA Blue Marble): 
  https://science.nasa.gov/earth/earth-observatory/the-blue-marble-true-color-global-imagery-at-1km-resolution/

> **Note:** The bounding box specified is approximately 10×10 km; degrees to kilometers vary with latitude (
> ~111.32 km per degree of latitude; longitude is scaled by cos(latitude)).


In [ ]:
# Install dependencies (Colab usually has earthengine-api preinstalled)
!pip -q install earthengine-api pillow matplotlib numpy scipy

import ee, requests, io
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
from scipy.ndimage import gaussian_filter


In [ ]:
# Authenticate & initialize Earth Engine (run once per session)
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()  # Follow link & paste auth code
    ee.Initialize()
print('Earth Engine initialized.')


## Parameters
Set the AOI bounding box, year, embedding RGB axes, visualization range, and whether to use NASA Blue Marble for the global panel.


In [ ]:
# --- AOI bounding box near Hamburg, IA ---
west, south, east, north = -95.70, 40.55, -95.60, 40.65
name = 'Small bounding box near Hamburg, IA - 10x10 km'

# Embedding year (2017 onward available)
YEAR = 2024

# Visualization stretch and RGB axes (choose any 3 of A00..A63)
VIS_MIN, VIS_MAX = -0.30, 0.30
BANDS_RGB = ['A01', 'A16', 'A09']

# Use NASA Blue Marble for global panel?
USE_BLUE_MARBLE = False
BLUE_MARBLE_URL = 'https://eoimages.gsfc.nasa.gov/images/imagerecords/73000/73909/world.topo.bathy.200407.3x5400x2700.jpg'

# AOI geometry
aoi = ee.Geometry.Rectangle([west, south, east, north])
aoi_bounds = aoi.bounds()
print(name)


## Load Satellite Embedding (64‑band) and prepare AOI image
Dataset: `GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL` (64 bands A00..A63).


In [ ]:
# Load annual Satellite Embeddings and mosaic the year
embeddings_ic = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
start = ee.Date.fromYMD(YEAR, 1, 1)
end   = start.advance(1, 'year')
embeddings_img = embeddings_ic.filter(ee.Filter.date(start, end)).mosaic()

# Clip to AOI and set visualization
embeddings_aoi = embeddings_img.clip(aoi)
vis_params = {'min': VIS_MIN, 'max': VIS_MAX, 'bands': BANDS_RGB}
print('Embeddings image prepared for year', YEAR)


## Helper: Download Earth Engine thumbnails as PIL images


In [ ]:
def ee_thumb_to_pil(image, params):
    url = image.getThumbURL(params)
    content = requests.get(url).content
    return Image.open(io.BytesIO(content))

# Extents
world_bbox = ee.Geometry.BBox(-180, -85, 180, 85)
# Retrieve AOI extent for plotting
coords = aoi_bounds.coordinates().getInfo()[0]
aoi_minlon = min(c[0] for c in coords)
aoi_maxlon = max(c[0] for c in coords)
aoi_minlat = min(c[1] for c in coords)
aoi_maxlat = max(c[1] for c in coords)
aoi_extent = (aoi_minlon, aoi_maxlon, aoi_minlat, aoi_maxlat)
world_extent = (-180, 180, -85, 85)
print('AOI extent:', aoi_extent)


## Build global & AOI images
- Global panel: embedding mosaic or optional NASA Blue Marble.
- AOI panel: embedding RGB clipped to the bounding box.


In [ ]:
# Global image
if USE_BLUE_MARBLE:
    global_img = Image.open(io.BytesIO(requests.get(BLUE_MARBLE_URL).content))
else:
    global_img = ee_thumb_to_pil(embeddings_img,
        {'min': VIS_MIN, 'max': VIS_MAX, 'bands': BANDS_RGB,
         'region': world_bbox, 'dimensions': 1024})

# AOI embeddings image
aoi_img = ee_thumb_to_pil(embeddings_aoi,
    {'min': VIS_MIN, 'max': VIS_MAX, 'bands': BANDS_RGB,
     'region': aoi_bounds, 'dimensions': 1024})

print('Global and AOI thumbnails ready.')


## Resilience class points
Load from CSV with columns `lon,lat,class` or generate placeholders inside the AOI for demonstration.


In [ ]:
import csv

USE_CSV = False   # set True and provide CSV_PATH to use your points
CSV_PATH = '/content/resilience_points.csv'

class_colors = {
    'Unaffected':       '#2ca02c',
    'Fully Resilient':  '#1f77b4',
    'Recovering':       '#ff7f0e',
    'Non-Resilient':    '#d62728',
}

points = []
if USE_CSV:
    with open(CSV_PATH, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            points.append((float(row['lon']), float(row['lat']), row['class']))
else:
    # Placeholder random points inside AOI bounds
    rng = np.random.default_rng(42)
    classes = list(class_colors.keys())
    for _ in range(50):
        lon_s = rng.uniform(aoi_minlon, aoi_maxlon)
        lat_s = rng.uniform(aoi_minlat, aoi_maxlat)
        cls = classes[rng.integers(0, len(classes))]
        points.append((lon_s, lat_s, cls))

print('Loaded points:', len(points))


## Plotting utilities: density shading, scale bar, north arrow


In [ ]:
def make_density(points, extent, res=350, sigma=6):
    minlon, maxlon, minlat, maxlat = extent
    xs = np.array([p[0] for p in points])
    ys = np.array([p[1] for p in points])
    xbins = np.linspace(minlon, maxlon, res)
    ybins = np.linspace(minlat, maxlat, res)
    H, _, _ = np.histogram2d(ys, xs, bins=[ybins, xbins])
    Hs = gaussian_filter(H, sigma)
    return Hs, xbins, ybins

def add_scale_bar(ax, length_km=2, xy=(0.08, 0.06), lw=3, color='k'):
    # Convert km to degrees of longitude at AOI latitude
    deg_lon = length_km / (111.32 * np.cos(np.deg2rad((aoi_minlat+aoi_maxlat)/2)))
    x0 = ax.get_xlim()[0] + (ax.get_xlim()[1] - ax.get_xlim()[0]) * xy[0]
    y0 = ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * xy[1]
    ax.plot([x0, x0 + deg_lon], [y0, y0], color=color, lw=lw)
    ax.text(x0 + deg_lon/2, y0 + (ax.get_ylim()[1]-ax.get_ylim()[0])*0.015,
            '{} km'.format(length_km), ha='center', va='bottom', fontsize=10, color=color)

def add_north_arrow(ax, xy=(0.95, 0.95), length=0.06, color='k'):
    x = ax.get_xlim()[0] + (ax.get_xlim()[1]-ax.get_xlim()[0])*xy[0]
    y = ax.get_ylim()[0] + (ax.get_ylim()[1]-ax.get_ylim()[0])*xy[1]
    ax.annotate('N', xy=(x, y),
                xytext=(x, y - length*(ax.get_ylim()[1]-ax.get_ylim()[0])),
                arrowprops=dict(facecolor=color, edgecolor=color, width=2, headwidth=10),
                ha='center', va='center', color=color, fontsize=12)

print('Utilities ready.')


## Compose the slide: Panels A, B, C
Saves a 300‑dpi PNG to your Colab runtime (`aoi_embeddings_resilience_slide.png`).


In [ ]:
# Density (for Panel C)
density, xbins, ybins = make_density(points, aoi_extent, res=320, sigma=6)

fig = plt.figure(figsize=(14, 6))
gs = fig.add_gridspec(1, 3, width_ratios=[1.2, 1.2, 1.6])

# Panel A: Global
axA = fig.add_subplot(gs[0, 0])
axA.imshow(global_img, extent=world_extent, origin='upper')
# Mark AOI center
center_lon = (aoi_minlon + aoi_maxlon)/2
center_lat = (aoi_minlat + aoi_maxlat)/2
axA.scatter([center_lon], [center_lat], s=80, facecolors='white', edgecolors='k', zorder=5)
axA.set_title('Global Context (AOI shown)', fontsize=12)
axA.set_xlabel('Longitude (°)')
axA.set_ylabel('Latitude (°)')
axA.grid(alpha=0.3, linestyle=':')
# Label coordinates
axA.text(center_lon+8, center_lat-15, '{:.2f}°, {:.2f}°'.format(center_lon, center_lat), color='white',
         bbox=dict(facecolor='black', alpha=0.5, pad=2), fontsize=10)

# Panel B: AOI Embedding RGB
axB = fig.add_subplot(gs[0, 1])
axB.imshow(aoi_img, extent=aoi_extent, origin='upper')
axB.scatter([center_lon], [center_lat], s=40, color='black', zorder=5)
axB.set_title('AOI Embedding RGB [{}], {}'.format(', '.join(BANDS_RGB), YEAR), fontsize=12)
axB.set_xlabel('Longitude (°)')
axB.set_ylabel('Latitude (°)')
axB.grid(alpha=0.3, linestyle=':')

# Dotted connector from Panel A to Panel B
con = ConnectionPatch(xyA=(center_lon, center_lat), coordsA=axA.transData,
                      xyB=(center_lon, center_lat), coordsB=axB.transData,
                      linestyle=':', color='k', lw=2)
fig.add_artist(con)

# Panel C: Points with density shading
axC = fig.add_subplot(gs[0, 2])
axC.imshow(density, extent=aoi_extent, origin='lower', cmap='Greys', alpha=0.25)
for cls, col in class_colors.items():
    cls_pts = [(x,y) for (x,y,c) in points if c == cls]
    if cls_pts:
        xs = [p[0] for p in cls_pts]
        ys = [p[1] for p in cls_pts]
        axC.scatter(xs, ys, label=cls, s=30, color=col, edgecolors='k', linewidths=0.3)

axC.set_title('Resilience Classes', fontsize=12)
axC.set_xlabel('Longitude (°)')
axC.set_ylabel('Latitude (°)')
axC.grid(alpha=0.2, linestyle=':')
add_scale_bar(axC, length_km=2)
add_north_arrow(axC)
leg = axC.legend(title='Resilience Classes', loc='lower right', frameon=True)
leg._legend_box.align = 'left'

plt.tight_layout()
plt.savefig('aoi_embeddings_resilience_slide.png', dpi=300)
plt.show()
print('Saved figure: aoi_embeddings_resilience_slide.png')


## (Optional) Export AOI embedding RGB to Google Drive
Exports the RGB embedding (selected bands) as a GeoTIFF clipped to the AOI. Earth Engine exports are asynchronous and go to Drive/Cloud/Asset.
See: https://book.geemap.org/chapters/07_data_export.html and EE export guides.


In [ ]:
task = ee.batch.Export.image.toDrive(**{
    'image': embeddings_aoi.select(BANDS_RGB),
    'description': 'AOI_Embeddings_RGB_{}'.format(YEAR),
    'folder': 'gee_exports',
    'region': aoi_bounds,
    'scale': 10,
    'maxPixels': 1e13,
    'fileFormat': 'GeoTIFF'
})
task.start()
print('Export started. Check Earth Engine Tasks dashboard.')


## Notes & Citations
- Earth Engine Python in Colab: https://developers.google.com/earth-engine/guides/python_install-colab
- Satellite Embedding dataset overview: https://developers.google.com/earth-engine/tutorials/community/satellite-embedding-01-introduction
- Hamburg, IA coordinates: https://en.wikipedia.org/wiki/Hamburg,_Iowa
- NASA Blue Marble (true color Earth): https://science.nasa.gov/earth/earth-observatory/the-blue-marble-true-color-global-imagery-at-1km-resolution/
